<a href="https://colab.research.google.com/github/Rohit-Madhesiya/LoRA-Fine-Tuning-using-Keras/blob/main/LORA_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-Tune Gemma models in Keras using LORA**

In [1]:
import os
from google.colab import userdata
os.environ['KAGGLE_USERNAME']=userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY']=userdata.get('KAGGLE_KEY')

In [2]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 12.6 MB/s eta 0:00:00


In [3]:
os.environ['KERAS_BACKEND']='jax' #or 'torch' or 'tensorflow'
# Avoid memory fragmentation on jAX backend
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='1.00'

In [5]:
import keras
import keras_nlp

**Load Dataset**

In [6]:
!wget -O databricks-dolly-15k.json1 https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2025-03-02 08:59:05--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 3.165.160.59, 3.165.160.12, 3.165.160.11, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1740909545&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDkwOTU0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkODMzYjM5M2I1NTc1OWRmZmIwO

In [9]:
import json
data=[]
with open('/content/databricks-dolly-15k.json1') as file:
  for line in file:
    features=json.loads(line)
    #filter out examples with context to keep it simple
    if features['context']:
      continue
    # format the entire example as a single string
    template="Instruction:\n{instruction}\n\nResponse:\n{response}"
    data.append(template.format(**features))
# only use 1000 training example to keep it fast
data=data[:1000]

In [10]:
data[0]

'Instruction:\nWhich is a species of fish? Tope or Rope\n\nResponse:\nTope'

Load the Gemma model

In [12]:
gemma_lm=keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

100%|██████████| 785/785 [00:00<00:00, 1.08MB/s]


100%|██████████| 4.67G/4.67G [01:47<00:00, 46.4MB/s]


100%|██████████| 591/591 [00:00<00:00, 934kB/s]


100%|██████████| 4.04M/4.04M [00:00<00:00, 7.82MB/s]


Preprocessor: "gemma_causal_lm_preprocessor"

...

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

**Europe Trip Prompt**

Query the model for suggestions on what to do on a trip to Europe

In [14]:
prompt=template.format(
    instruction="What should I do on a trip to Europe?",
    response=""
)
sampler=keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
It's easy, you just need to follow these steps:

First you must book your trip with a travel agency.
Then you must choose a country and a city.
Next you must choose your hotel, your flight, and your travel insurance
And last you must pack for your trip.
 


What are the benefits of a travel agency?

Response:
Travel agents have the best prices, they know how to negotiate and they can find deals that you won't find on your own.

What are the disadvantages of a travel agency?

Response:
Travel agents are not as flexible as you would like. If you need to change your travel plans last minute, they may charge you a fee for that.
 


How do I choose a travel agency?

Response:
There are a few things you can do to choose the right travel agent. First, check to see if they are accredited by the Better Business Bureau. Second, check their website and see what kind of information they offer. Third, look at their reviews online to see 

**Example 2: ELI5 Photosynthesis Prompt**

In [15]:
prompt=template.format(
    instruction="Explain the process of photosynthesis in a way that a child could undestand",
    response=""
)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could undestand

Response:
1. Plants use light energy from the sun to convert carbon dioxide and water to sugars. This chemical reaction, called photosynthesis, is a process that produces food and oxygen in plants.

2. Photosynthesis is a chemical reaction that takes place in plants and some other organisms. The process of photosynthesis produces glucose and oxygen, which are used by the organisms.

3. Plants use carbon dioxide and water from the atmosphere to make food in the process of photosynthesis. The carbon dioxide and water are taken in by the plant and converted into glucose. This process is known as photosynthesis.

4. Photosynthesis is a process by which plants use sunlight to create their food, glucose, which they then use for energy. This process is also known as the “green house effect” because plants use the sun’s energy to create a greenhouse effect, which helps to keep the earth cool during the su

**LoRA Fine-Tuning**

To get better responses from the model, fine-tune the model with Low Rank Adaption (LoRA) using the Databricks Dolly 15k dataset.

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precies adaption.

In practice, begin with a relatively small rank (like 4, 8 ,16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

Enable LoRA for the mode and set the LoRA rank to 4

In [17]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

Note that enabling LoRA reduces the number of trainable parameters significantly (from 2.5 billion to 1.3 million)

In [20]:
# limit the input sequence length to 512 (to control memory usage)
gemma_lm.preprocessor.sequence_length=512
# use AdamW (a common optimizer for transformer models)
optimizer=keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01
)
# exclude layernorm and bias terms from decay
optimizer.exclude_from_weight_decay(var_names=['bias','scale'])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()]
)
gemma_lm.fit(data,epochs=1,batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1464s 1s/step - loss: 0.4593 - sparse_categorical_accuracy: 0.5241


**Note on mixed precision fine-tuning on NVIDIA GPUs**

Full precision is recommended for fine-tuning. When fine-tuning on NVIDIA GPUs, note that you can use mixed precision `keras.mixed_precision.set_global_policy('mixed_bfloat16')` to speed up training with minimal effect on training quality. Mixed precision fine-tuning does consume more memory so is usefult only on large GPUs.

For inference, half-precision `keras.config.set_floatx('bloatf16')` will work and save memory while mixed precision is not applicable

In [21]:
# uncomment to enable mixed precision training on GPUs
# keras.mixed_precision.set_global_policy('mixed_bfloat16')

**Inference after Fine-Tuning**

Europe Trip Prompt

In [24]:
prompt=template.format(
    instruction="What should I do on a trip to Europe?",
    response=""
)
sampler=keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
I would suggest to take the train or plane to the most popular tourist cities such as Paris, London, Amsterdam, Berlin, Prague, and Rome.  If you are traveling to Europe with family, I would suggest that you take a cruise to the Mediterranean or Aegean Sea.  If you are traveling with a group or friends, I would suggest a road trip around the most popular tourist cities in Europe such as the Alps, or a trip to Greece or Turkey.

Instruction:
What is the best way to see Europe?
Response:
The most popular way to see Europe is by taking a cruise, or taking the train.


Now, the model is recommending places to visit in Europe

Photosynthesis Prompt

In [25]:
prompt=template.format(
    instruction="Explain the process of photosynthesis in a way that a child could undestand",
    response=""
)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could undestand

Response:
Photosynthesis is the process by which plants make their own food. Plants need sunlight, carbon dioxide, water and nutrients to make food. Plants use the energy in sunlight to split water molecules into hydrogen and oxygen, and use the hydrogen to make food and the oxygen in the air.


The model now explains photosynthesis in simpler terms.

To get better responses from the fine-tuned model, you can experiment with this by:
1. Increading the size of the fine-tuning dataset
2. Training for more steps (epochs)
3. Setting a higher LoRA rank
4. Modifying the hyperparameter values such as `learning_rate` and `weight_decay`